In [1]:
!pip install transformers

In [2]:
!pip install datasets

  Using cached huggingface_hub-0.0.19-py3-none-any.whl (56 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.10.1
    Uninstalling huggingface-hub-0.10.1:
      Successfully uninstalled huggingface-hub-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.2.2 requires huggingface-hub>=0.4.0, but you have huggingface-hub 0.0.19 which is incompatible.


In [3]:
pip install tensorflow-datasets -U

Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow
import numpy
import transformers
import argparse

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(argparse.__version__)

2.6.0
1.21.4
4.11.3
1.1


In [5]:
import os
import csv
import warnings
import pandas as pd
import numpy as np
from argparse import ArgumentParser
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, AutoConfig
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures

print("슝~")

슝~


In [6]:
pdata_train = pd.read_parquet('/aiffel/aiffel/parquetfiles/dataset_shard_train.parquet')
pdata_validation = pd.read_parquet('/aiffel/aiffel/parquetfiles/dataset_shard_validation.parquet')
pdata_test = pd.read_parquet('/aiffel/aiffel/parquetfiles/dataset_shard_test.parquet')

In [7]:
pdata_train.head()

,premise,hypothesis,label,idx
0,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,1,0
1,you know during the season and i guess at at y...,You lose the things to the following level if ...,0,1
2,One of our number will carry out your instruct...,A member of my team will execute your orders w...,0,2
3,How do you know? All this is their information...,This information belongs to them.,0,3
4,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,1,4


In [8]:
pdata_train.shape

(392702, 4)

In [9]:
df = pd.DataFrame(pdata_train)
dataset_train = df.truncate(before=0,after=9999,axis=0)
dataset_train.shape

(10000, 4)

In [10]:
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        Gets an example from a dict with tensorflow tensors.

        Args:
            tensor_dict: Keys and values should match the corresponding Glue
                tensorflow_dataset examples.
        """
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def tfds_map(self, example):
        """
        Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are. This method converts
        examples to the correct format.
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

print("슝~")

슝~


In [11]:
class MnliProcessor(DataProcessor):
    """Processor for the MultiNLI data set (GLUE version)."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8")
            )

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev_matched.tsv")), "dev_matched")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test_matched.tsv")), "test_matched")

    def get_labels(self):
        """See base class."""
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for i, line in enumerate(lines):
            if i == 0:
                continue
            guid = f"{set_type}-{line[0]}"
            text_a = line[8]
            text_b = line[9]
            label = None if set_type.startswith("test") else line[-1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

In [12]:
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: %s" % (example.guid))
        print("features: %s" % features[i])

    return features

print("슝~")

슝~


In [13]:
def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
        
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )

print("슝~")

슝~


In [14]:
import datasets
from datasets import load_dataset

huggingface_mnli_dataset = load_dataset('glue', 'mnli')
print(huggingface_mnli_dataset)

Reusing dataset glue (/aiffel/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})


In [15]:
!pip install sentence_transformers

  Using cached huggingface_hub-0.10.1-py3-none-any.whl (163 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 1.14.0 requires huggingface-hub<0.1.0,>=0.0.19, but you have huggingface-hub 0.10.1 which is incompatible.


In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("symanto/mpnet-base-snli-mnli")

model = AutoModelForSequenceClassification.from_pretrained("symanto/mpnet-base-snli-mnli")

In [17]:
data, info = tfds.load('glue/mnli', with_info=True)
info.splits['train'].num_examples

INFO:absl:Load dataset info from /aiffel/tensorflow_datasets/glue/mnli/2.0.0
INFO:absl:Field info.release_notes from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.splits from disk and from code do not match. Keeping the one from code.
INFO:absl:Reusing dataset glue (/aiffel/tensorflow_datasets/glue/mnli/2.0.0)
INFO:absl:Constructing tf.data.Dataset glue for split None, from /aiffel/tensorflow_datasets/glue/mnli/2.0.0


392702

In [18]:
data

{'train': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'validation_matched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'validation_mismatched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'test_matched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'test_mismatched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>}

In [19]:
examples = data['train'].take(1)
for example in examples:
    sentence1 = example['premise']
    sentence2 = example['hypothesis']
    label = example['label']
    print(sentence1)
    print(sentence2)
    print(label)

tf.Tensor(b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', shape=(), dtype=string)
tf.Tensor(b'Meaningful partnerships with stakeholders is crucial.', shape=(), dtype=string)
tf.Tensor(1, shape=(), dtype=int64)


In [20]:
processor = MnliProcessor()
num_classes = len(processor.get_labels())

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [21]:
#del model
#del train_dataset_batch
#del validation_dataset_batch
#del test_dataset_batch

In [22]:
from transformers import Trainer, TrainingArguments
output_dir = os.getenv('HOME')+'/aiffel/transformers'
metric_name = 'accuracy'

training_arguments = TrainingArguments(
    output_dir, # output이 저장될 경로
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, #learning_rate
    per_device_train_batch_size = 16, # 각 device 당 batch size
    per_device_eval_batch_size = 16, # evaluation 시에 batch size
    num_train_epochs = 3, # train 시킬 총 epochs
    weight_decay = 0.01, # weight decay
    max_steps = 60
)

In [23]:
from datasets import load_metric
metric = load_metric('glue', 'mnli')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

In [24]:
import numpy as np
from datasets import load_metric

max_length = 128

def get_basic_example_fn(tokenizer, src_cols=[], tar_col='label', label_fn=None, max_src_len=128, truncation=True, padding=max_length):

    def example_fn(examples):
        output = tokenizer(*[examples[col] for col in [c for c in src_cols if c in examples]],
                           padding=padding,
                           max_length=max_src_len,
                           truncation=True)
        if tar_col in examples:
            output["labels"] = [label_fn(c) for c in examples[tar_col]] if label_fn else examples[tar_col]
        return output
    
    return example_fn

metric_fn = load_metric('glue', 'mnli')

def metric(p):
    preds, labels = p
    if not isinstance(preds, tuple) and not isinstance(preds, list):
        if len(preds.shape) == 2 and preds.shape[1] == 1:
            preds = preds[:, 0]
        elif len(preds.shape) - len(labels.shape) == 1:
            preds = np.argmax(preds, axis=-1)
    return metric_fn.compute(predictions=preds, references=labels)

In [25]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments, 
    DataCollatorWithPadding, 
    EarlyStoppingCallback
)

TRAIN = "/aiffel/aiffel/parquetfiles/dataset_shard_train.parquet"
TEST = "/aiffel/aiffel/parquetfiles/dataset_shard_test.parquet"
VALIDATION = "/aiffel/aiffel/parquetfiles/dataset_shard_validation.parquet"
CKPT = "runs/"
MODEL = 'symanto/mpnet-base-snli-mnli'

MAX_LEN = 128
TRAIN_BATCH = 8
EVAL_BATCH = 8

def transform(data):
    return tokenizer(
      data['premise'],
      data['hypothesis'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = False,
      )

# Prepare the train/eval/test dataset.
map_dict = {'contradiction': 0, 'neutral': 1, 'entailment': 2}

tokenizer = AutoTokenizer.from_pretrained(MODEL)

example_fn = get_basic_example_fn(
    tokenizer,
    src_cols=['premise', 'hypothesis'], 
    tar_col='label', 
    label_fn=lambda x: map_dict.get(x),
    max_src_len=MAX_LEN,
    truncation=True, 
    padding="max_length"
    )


dataset = load_dataset("parquet", data_files={"train":VALIDATION})
dataset = dataset['train'].train_test_split(0.02)
encoded_dataset_train = dataset['test'].map(transform, batched=True)
dataset_2 = load_dataset("parquet", data_files={"eval":VALIDATION})
encoded_dataset_eval = dataset_2['eval'].map(transform, batched=True)



# Define the model, arguments, and Trainer with huggingface transformers.
training_arguments = TrainingArguments(
    output_dir="runs/roberta_basic/",
    num_train_epochs=5,
    per_device_train_batch_size=TRAIN_BATCH,
    per_device_eval_batch_size=EVAL_BATCH,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_steps=1000,
    save_total_limit=3,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    load_best_model_at_end=True,
    label_smoothing_factor=0.0,
)

model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=len(map_dict))

trainer = Trainer(
    model=model,
    args=training_arguments,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=metric,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_eval,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_virtual_device_configuration(
           gpus[0],
           [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)
# Training.

trainer.train()

# Prepare the test data.
test_data = load_dataset("parquet", data_files={"test":TEST})
encoded_dataset_test = test_data['test'].map(transform, batched=True)
# Predict the test outputs.
outputs = trainer.predict(encoded_dataset_test)

# Transform the test outputs for the submission.
df_sub = pd.DataFrame({"index":encoded_dataset_test['index'], "label":np.argmax(outputs.predictions, axis=-1)})
df_sub['label'] = df_sub['label'].apply(lambda x: {v:k for k,v in map_dict.items()}[x])
df_sub.to_csv("submission.csv", index=False)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

The following columns in the training set  don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: idx, hypothesis, premise.
***** Running training *****
  Num examples = 197
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 125


Virtual devices cannot be modified after being initialized


Epoch,Training Loss,Validation Loss,Accuracy
1,0.532000,0.594419,0.832501
2,0.178200,0.725146,0.836169


The following columns in the evaluation set  don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: idx, hypothesis, premise.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 8
Saving model checkpoint to runs/roberta_basic/checkpoint-25
Configuration saved in runs/roberta_basic/checkpoint-25/config.json
Model weights saved in runs/roberta_basic/checkpoint-25/pytorch_model.bin
tokenizer config file saved in runs/roberta_basic/checkpoint-25/tokenizer_config.json
Special tokens file saved in runs/roberta_basic/checkpoint-25/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: idx, hypothesis, premise.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 8
Saving model checkpoint to runs/roberta_basic/checkpoint-50
Configuration saved in runs/roberta_basic/checkpoint-50/config.json
Model weig

KeyboardInterrupt: 